In [37]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import config

# 最大表示列数の指定（ここでは50列を指定）
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)


def make_dir(dir_name: str):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# 型のエンコーディング
## df｜df_MB_dtype_encoded
順序尺度特徴量とboolean特徴量について、データを扱いやすい形に変換する。

In [38]:
df_MB_dtype_encoded = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_DIR + "/2.0.1-df_MB_dropped.pkl"
)
df_MB_dtype_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1124 entries, 1 to 1743
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PATIENT_ID                     1124 non-null   object 
 1   LYMPH_NODES_EXAMINED_POSITIVE  1124 non-null   float64
 2   NPI                            1124 non-null   float64
 3   CELLULARITY                    1124 non-null   object 
 4   CHEMOTHERAPY                   1124 non-null   object 
 5   COHORT                         1124 non-null   float64
 6   ER_IHC                         1124 non-null   object 
 7   HER2_SNP6                      1124 non-null   object 
 8   HORMONE_THERAPY                1124 non-null   object 
 9   INFERRED_MENOPAUSAL_STATE      1124 non-null   object 
 10  SEX                            1124 non-null   object 
 11  INTCLUST                       1124 non-null   object 
 12  AGE_AT_DIAGNOSIS               1124 non-null   f

In [39]:
[
    str(column) + str(df_MB_dtype_encoded[column].unique())
    for column in df_MB_dtype_encoded.columns
]

["PATIENT_ID['MB-0002' 'MB-0008' 'MB-0010' ... 'MB-6237' 'MB-6239' 'MB-6346']",
 'LYMPH_NODES_EXAMINED_POSITIVE[ 0.  8.  1.  3. 16.  5. 14.  6.  2.  4.  9. 22.  7. 21. 12. 25. 10. 41.\n 15. 13. 11. 19. 17. 18. 23. 26. 31.]',
 'NPI[4.02  6.08  4.062 5.032 3.056 3.044 4.066 4.046 4.032 4.078 3.068 5.08\n 4.14  2.054 6.3   5.034 3.06  4.05  4.12  5.052 5.06  3.04  4.1   2.018\n 3.07  3.028 6.048 2.034 3.16  5.044 5.07  3.026 3.036 4.06  4.054 3.032\n 5.076 4.038 6.1   4.024 3.024 4.028 6.104 5.048 4.034 3.052 4.04  6.088\n 4.048 6.072 4.088 6.068 4.042 3.05  6.096 2.026 5.042 6.056 6.036 5.038\n 5.05  2.02  3.006 3.08  5.04  6.04  3.038 3.034 6.07  2.092 4.044 4.03\n 4.022 4.052 5.11  3.018 5.046 6.058 6.02  3.042 2.094 3.066 5.064 4.09\n 4.058 4.056 6.032 2.044 4.026 3.1   4.036 5.134 5.1   3.03  6.062 6.094\n 6.044 6.05  5.056 6.086 6.03  6.106 2.022 6.084 6.09  5.03  6.36  4.08\n 5.062 6.12  4.084 4.114 5.066 2.032 6.2   5.068 6.026 5.14  5.078 5.16\n 4.064 6.038 3.058 6.06  3.046 3.02

In [40]:
target_columns = [
    # patient
    "OS_MONTHS",
    "RFS_MONTHS",
    "OS_STATUS",
    "RFS_STATUS",
    "VITAL_STATUS",
]

int_columns = [
    # patient
    "LYMPH_NODES_EXAMINED_POSITIVE",
    "OS_MONTHS",
    "RFS_MONTHS",
]

float_columns = [
    # patient
    "NPI",
    "AGE_AT_DIAGNOSIS",
    # sample
    "TUMOR_SIZE",
    "TMB_NONSYNONYMOUS",
]

str_columns = []

# 質的変数（順序尺度）
qualitative_ordinal_columns = [
    # patient
    "CELLULARITY",
    "ER_IHC",
    "HER2_SNP6",
    "INFERRED_MENOPAUSAL_STATE",
    # sample
    "ER_STATUS",
    "HER2_STATUS",
    "GRADE",
    "PR_STATUS",
    "TUMOR_STAGE",
]
# 質的変数（名義尺度）
qualitative_name_columns = [
    # patient
    "COHORT",
    "INTCLUST",
    "CLAUDIN_SUBTYPE",
    "THREEGENE",
    "HISTOLOGICAL_SUBTYPE",
    "BREAST_SURGERY",
    "LATERALITY",
    "VITAL_STATUS",
    # sample
    "CANCER_TYPE",
    "CANCER_TYPE_DETAILED",
    "ONCOTREE_CODE",
]

bool_columns = [
    # patient
    "CHEMOTHERAPY",
    "HORMONE_THERAPY",
    "RADIO_THERAPY",
    "OS_STATUS",
    "RFS_STATUS",
]

meanless_columns = [
    # patient
    "PATIENT_ID",
    "SEX",
    # sample
    #'PATIENT_ID',
    "SAMPLE_ID",
    "SAMPLE_TYPE",
]

print(
    df_MB_dtype_encoded.shape[1],
    len(int_columns)
    + len(float_columns)
    + len(ｓｔｒ_columns)
    + len(qualitative_ordinal_columns)
    + len(qualitative_name_columns)
    + len(bool_columns)
    + len(meanless_columns),
)
assert df_MB_dtype_encoded.shape[1] == len(int_columns) + len(float_columns) + len(
    ｓｔｒ_columns
) + len(qualitative_ordinal_columns) + len(qualitative_name_columns) + len(
    bool_columns
) + len(
    meanless_columns
), "lack or too much columns"

36 36


# 質的変数（順序尺度）のエンコーディング
qualitative_ordinal_columnsは、順序のあるstrの変数で構成されている。
そこで大小関係に対応するようstrをintに変換する。

In [41]:
df_MB_dtype_encoded[qualitative_ordinal_columns].head()

,CELLULARITY,ER_IHC,HER2_SNP6,INFERRED_MENOPAUSAL_STATE,ER_STATUS,HER2_STATUS,GRADE,PR_STATUS,TUMOR_STAGE
1,High,Positve,NEUTRAL,Pre,Positive,Negative,3.0,Positive,1.0
4,High,Positve,NEUTRAL,Post,Positive,Negative,3.0,Positive,2.0
5,Moderate,Positve,NEUTRAL,Post,Positive,Negative,3.0,Positive,4.0
10,Moderate,Positve,GAIN,Post,Positive,Negative,3.0,Negative,2.0
11,High,Negative,LOSS,Post,Positive,Negative,2.0,Negative,2.0


In [42]:
for column in qualitative_ordinal_columns:
    print(column)
    display(df_MB_dtype_encoded[column].value_counts())
    print("-----" * 10)

CELLULARITY


High        564
Moderate    428
Low         132
Name: CELLULARITY, dtype: int64

--------------------------------------------------
ER_IHC


Positve     875
Negative    249
Name: ER_IHC, dtype: int64

--------------------------------------------------
HER2_SNP6


...

--------------------------------------------------
PR_STATUS


Positive    584
Negative    540
Name: PR_STATUS, dtype: int64

--------------------------------------------------
TUMOR_STAGE


2.0    640
1.0    381
3.0     95
4.0      8
Name: TUMOR_STAGE, dtype: int64

--------------------------------------------------


In [43]:
df_MB_dtype_encoded["CELLULARITY"].replace(
    {"High": 3, "Moderate": 2, "Low": 1}, inplace=True
)
df_MB_dtype_encoded["ER_IHC"].replace({"Positve": 1, "Negative": -1}, inplace=True)
df_MB_dtype_encoded["HER2_SNP6"].replace(
    {"GAIN": 3, "NEUTRAL": 2, "LOSS": 1, "UNDEF": 0}, inplace=True
)
df_MB_dtype_encoded["INFERRED_MENOPAUSAL_STATE"].replace(
    {"Post": 1, "Pre": -1}, inplace=True
)
df_MB_dtype_encoded["ER_STATUS"].replace({"Positive": 1, "Negative": -1}, inplace=True)
df_MB_dtype_encoded["HER2_STATUS"].replace(
    {"Positive": 1, "Negative": -1}, inplace=True
)
# df_MB_dtype_encoded['GRADEGRADE']
df_MB_dtype_encoded["PR_STATUS"].replace({"Positive": 1, "Negative": -1}, inplace=True)
# df_MB_dtype_encoded['TUMOR_STAGE']

for column in qualitative_ordinal_columns:
    print(column)
    display(df_MB_dtype_encoded[column].value_counts())
    print("-----" * 10)

CELLULARITY


3    564
2    428
1    132
Name: CELLULARITY, dtype: int64

--------------------------------------------------
ER_IHC


 1    875
-1    249
Name: ER_IHC, dtype: int64

--------------------------------------------------
HER2_SNP6


...

--------------------------------------------------
PR_STATUS


 1    584
-1    540
Name: PR_STATUS, dtype: int64

--------------------------------------------------
TUMOR_STAGE


2.0    640
1.0    381
3.0     95
4.0      8
Name: TUMOR_STAGE, dtype: int64

--------------------------------------------------


# boolのエンコーディング
bool_columnsは対象的な2項目から構成されているため、これらの項目をboolean型にする。
Yes、Noのみを対象とする（2項目のみから構成される特徴量でも、Yes、No以外の場合は3項目を考慮してカテゴリ特徴量として考える）。

In [44]:
df_MB_dtype_encoded[bool_columns].astype(bool).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1124 entries, 1 to 1743
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   CHEMOTHERAPY     1124 non-null   bool 
 1   HORMONE_THERAPY  1124 non-null   bool 
 2   RADIO_THERAPY    1124 non-null   bool 
 3   OS_STATUS        1124 non-null   bool 
 4   RFS_STATUS       1124 non-null   bool 
dtypes: bool(5)
memory usage: 14.3 KB


In [45]:
for column in bool_columns:
    print(column)
    display(df_MB_dtype_encoded[column].value_counts())
    print("-----" * 10)

CHEMOTHERAPY


NO     873
YES    251
Name: CHEMOTHERAPY, dtype: int64

--------------------------------------------------
HORMONE_THERAPY


YES    697
NO     427
Name: HORMONE_THERAPY, dtype: int64

--------------------------------------------------
RADIO_THERAPY


...

--------------------------------------------------
OS_STATUS


1:DECEASED    620
0:LIVING      504
Name: OS_STATUS, dtype: int64

--------------------------------------------------
RFS_STATUS


0:Not Recurred    659
1:Recurred        465
Name: RFS_STATUS, dtype: int64

--------------------------------------------------


In [46]:
df_MB_dtype_encoded["CHEMOTHERAPY"].replace({"YES": True, "NO": False}, inplace=True)
df_MB_dtype_encoded["HORMONE_THERAPY"].replace({"YES": True, "NO": False}, inplace=True)
df_MB_dtype_encoded["RADIO_THERAPY"].replace({"YES": True, "NO": False}, inplace=True)
df_MB_dtype_encoded["OS_STATUS"].replace(
    {"1:DECEASED": True, "0:LIVING": False}, inplace=True
)
df_MB_dtype_encoded["RFS_STATUS"].replace(
    {"1:Recurred": True, "0:Not Recurred": False}, inplace=True
)

for column in bool_columns:
    print(column)
    display(df_MB_dtype_encoded[column].value_counts())
    print("-----" * 10)

CHEMOTHERAPY


False    873
True     251
Name: CHEMOTHERAPY, dtype: int64

--------------------------------------------------
HORMONE_THERAPY


True     697
False    427
Name: HORMONE_THERAPY, dtype: int64

--------------------------------------------------
RADIO_THERAPY


...

--------------------------------------------------
OS_STATUS


True     620
False    504
Name: OS_STATUS, dtype: int64

--------------------------------------------------
RFS_STATUS


False    659
True     465
Name: RFS_STATUS, dtype: int64

--------------------------------------------------


In [47]:
df_MB_dtype_encoded[bool_columns].dtypes

CHEMOTHERAPY       bool
HORMONE_THERAPY    bool
RADIO_THERAPY      bool
OS_STATUS          bool
RFS_STATUS         bool
dtype: object

In [48]:
# save
make_dir(config.INTERIM_PICKLE_PREPROCESSED_DIR)
df_MB_dtype_encoded.to_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_DIR + "/2.0.2-df_MB_dtype_encoded.pkl"
)

# Onehot encoding
## df｜df_MB_onehot_encoded(onehot encodingしたdf), df_MB_onehot_concated(onehot encodingしたdfと元のdfの結合)
上記で作成した一部特徴量を修正したdfを元に、onehot encodingを行う。
onehot encoding自体は名義尺度の質的変数カラムであるqualitative_name_columnsのみに対してのみ実施するが、最終的に特徴量の型の修正はモデルへ入力する際に必要になるので、df_MB_dtype_encodedを元にonehot encodingを実施する。    

In [58]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import config

# 最大表示列数の指定（ここでは50列を指定）
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)


def make_dir(dir_name: str):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [59]:
df_MB_dtype_encoded = pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_DIR + "/2.0.2-df_MB_dtype_encoded.pkl"
)
df_MB_dtype_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1124 entries, 1 to 1743
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PATIENT_ID                     1124 non-null   object 
 1   LYMPH_NODES_EXAMINED_POSITIVE  1124 non-null   float64
 2   NPI                            1124 non-null   float64
 3   CELLULARITY                    1124 non-null   int64  
 4   CHEMOTHERAPY                   1124 non-null   bool   
 5   COHORT                         1124 non-null   float64
 6   ER_IHC                         1124 non-null   int64  
 7   HER2_SNP6                      1124 non-null   int64  
 8   HORMONE_THERAPY                1124 non-null   bool   
 9   INFERRED_MENOPAUSAL_STATE      1124 non-null   int64  
 10  SEX                            1124 non-null   object 
 11  INTCLUST                       1124 non-null   object 
 12  AGE_AT_DIAGNOSIS               1124 non-null   f

In [60]:
# VITAL_STATUSは目的変数の生成に必要&リークになりうるので、onehot encodingからは除外する
qualitative_name_columns.remove(
    "VITAL_STATUS"
) if "VITAL_STATUS" in qualitative_name_columns else qualitative_name_columns
qualitative_name_columns

['COHORT',
 'INTCLUST',
 'CLAUDIN_SUBTYPE',
 'THREEGENE',
 'HISTOLOGICAL_SUBTYPE',
 'BREAST_SURGERY',
 'LATERALITY',
 'CANCER_TYPE',
 'CANCER_TYPE_DETAILED',
 'ONCOTREE_CODE']

In [61]:
# 多重共線性回避のために、drop='first'
onehot_encoder = OneHotEncoder(drop="first")
onehot_encoding_columns = qualitative_name_columns
onehot_encoder.fit(df_MB_dtype_encoded[onehot_encoding_columns])
display(onehot_encoder.categories_)
display(onehot_encoder.get_feature_names_out(onehot_encoding_columns))
# enc.transform(df_MB_onehot_encoded[encoding_columns]).toarray()
df_MB_onehot_encoded = pd.DataFrame(
    onehot_encoder.transform(df_MB_dtype_encoded[onehot_encoding_columns]).toarray(),
    columns=onehot_encoder.get_feature_names_out(onehot_encoding_columns),
).astype(int)
df_MB_onehot_encoded.shape

[array([1., 2., 3., 5.]),
 array(['1', '10', '2', '3', '4ER+', '4ER-', '5', '6', '7', '8', '9'],
       dtype=object),
 array(['Basal', 'Her2', 'LumA', 'LumB', 'NC', 'Normal', 'claudin-low'],
       dtype=object),
 array(['ER+/HER2- High Prolif', 'ER+/HER2- Low Prolif', 'ER-/HER2-',
        'HER2+'], dtype=object),
 array(['Ductal/NST', 'Lobular', 'Medullary', 'Mixed', 'Mucinous', 'Other',
        'Tubular/ cribriform'], dtype=object),
 array(['BREAST CONSERVING', 'MASTECTOMY'], dtype=object),
 array(['Left', 'Right'], dtype=object),
 array(['Breast Cancer'], dtype=object),
 array(['Breast', 'Breast Invasive Ductal Carcinoma',
        'Breast Invasive Lobular Carcinoma',
        'Breast Invasive Mixed Mucinous Carcinoma',
        'Breast Mixed Ductal and Lobular Carcinoma'], dtype=object),
 array(['BREAST', 'IDC', 'ILC', 'IMMC', 'MDLC'], dtype=object)]

array(['COHORT_2.0', 'COHORT_3.0', 'COHORT_5.0', 'INTCLUST_10',
       'INTCLUST_2', 'INTCLUST_3', 'INTCLUST_4ER+', 'INTCLUST_4ER-',
       'INTCLUST_5', 'INTCLUST_6', 'INTCLUST_7', 'INTCLUST_8',
       'INTCLUST_9', 'CLAUDIN_SUBTYPE_Her2', 'CLAUDIN_SUBTYPE_LumA',
       'CLAUDIN_SUBTYPE_LumB', 'CLAUDIN_SUBTYPE_NC',
       'CLAUDIN_SUBTYPE_Normal', 'CLAUDIN_SUBTYPE_claudin-low',
       'THREEGENE_ER+/HER2- Low Prolif', 'THREEGENE_ER-/HER2-',
       'THREEGENE_HER2+', 'HISTOLOGICAL_SUBTYPE_Lobular',
       'HISTOLOGICAL_SUBTYPE_Medullary', 'HISTOLOGICAL_SUBTYPE_Mixed',
       'HISTOLOGICAL_SUBTYPE_Mucinous', 'HISTOLOGICAL_SUBTYPE_Other',
       'HISTOLOGICAL_SUBTYPE_Tubular/ cribriform',
       'BREAST_SURGERY_MASTECTOMY', 'LATERALITY_Right',
       'CANCER_TYPE_DETAILED_Breast Invasive Ductal Carcinoma',
       'CANCER_TYPE_DETAILED_Breast Invasive Lobular Carcinoma',
       'CANCER_TYPE_DETAILED_Breast Invasive Mixed Mucinous Carcinoma',
       'CANCER_TYPE_DETAILED_Breast Mixed Ducta

(1124, 38)

In [62]:
df_MB_onehot_encoded.head()

,COHORT_2.0,COHORT_3.0,COHORT_5.0,INTCLUST_10,INTCLUST_2,INTCLUST_3,INTCLUST_4ER+,INTCLUST_4ER-,INTCLUST_5,INTCLUST_6,INTCLUST_7,INTCLUST_8,INTCLUST_9,CLAUDIN_SUBTYPE_Her2,CLAUDIN_SUBTYPE_LumA,CLAUDIN_SUBTYPE_LumB,CLAUDIN_SUBTYPE_NC,CLAUDIN_SUBTYPE_Normal,CLAUDIN_SUBTYPE_claudin-low,THREEGENE_ER+/HER2- Low Prolif,THREEGENE_ER-/HER2-,THREEGENE_HER2+,HISTOLOGICAL_SUBTYPE_Lobular,HISTOLOGICAL_SUBTYPE_Medullary,HISTOLOGICAL_SUBTYPE_Mixed,HISTOLOGICAL_SUBTYPE_Mucinous,HISTOLOGICAL_SUBTYPE_Other,HISTOLOGICAL_SUBTYPE_Tubular/ cribriform,BREAST_SURGERY_MASTECTOMY,LATERALITY_Right,CANCER_TYPE_DETAILED_Breast Invasive Ductal Carcinoma,CANCER_TYPE_DETAILED_Breast Invasive Lobular Carcinoma,CANCER_TYPE_DETAILED_Breast Invasive Mixed Mucinous Carcinoma,CANCER_TYPE_DETAILED_Breast Mixed Ductal and Lobular Carcinoma,ONCOTREE_CODE_IDC,ONCOTREE_CODE_ILC,ONCOTREE_CODE_IMMC,ONCOTREE_CODE_MDLC
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0


# 元のdfとonehotしたdfの結合
dfの形などを調査

2つのdfのshapeを確認

In [63]:
df_MB_dtype_encoded.shape, df_MB_onehot_encoded.shape

((1124, 36), (1124, 38))

In [64]:
# 単に結合しただけでは形が異なる
pd.concat([df_MB_dtype_encoded, df_MB_onehot_encoded], axis=1).shape

(1481, 74)

In [65]:
# concatがうまく行かない理由はindexの順番。
# 元のdfのnullをdropしたため。

# なのでindexをresetしてからconcatする

# onehotで生成した元のカラムの削除
df_MB_onehot_concated = pd.concat(
    [
        df_MB_dtype_encoded.reset_index(drop=True),
        df_MB_onehot_encoded.reset_index(drop=True),
    ],
    axis=1,
).drop(onehot_encoding_columns, axis=1)
display(df_MB_onehot_concated.shape)

display(df_MB_onehot_concated.info())

(1124, 64)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 64 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   PATIENT_ID                                                      1124 non-null   object 
 1   LYMPH_NODES_EXAMINED_POSITIVE                                   1124 non-null   float64
 2   NPI                                                             1124 non-null   float64
 3   CELLULARITY                                                     1124 non-null   int64  
 4   CHEMOTHERAPY                                                    1124 non-null   bool   
 5   ER_IHC                                                          1124 non-null   int64  
 6   HER2_SNP6                                                       1124 non-null   int64  
 7   HORMONE_THERAPY                                    

None

In [66]:
# save
make_dir(config.INTERIM_PICKLE_PREPROCESSED_DIR)
df_MB_onehot_concated.to_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_DIR + "/2.0.2-df_MB_onehot_concated.pkl"
)